In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
def tileset_info():
    return {
        'min_pos': [-2.5, -2.5],
        'max_pos': [2.5, 2.5],
        'bins_per_dimension': 256,
        'max_width': 5,
        'max_zoom': 20
    }

In [33]:
from numba import jit, vectorize, guvectorize, float64, complex64, int32, float32
import numpy as np

# from: https://www.ibm.com/developerworks/community/blogs/jfp/entry/How_To_Compute_Mandelbrodt_Set_Quickly?lang=en

@jit(int32(complex64, int32))
def mandelbrot(c,maxiter):
    nreal = 0
    real = 0
    imag = 0
    for n in range(maxiter):
        nreal = real*real - imag*imag + c.real
        imag = 2* real*imag + c.imag
        real = nreal;
        if real * real + imag * imag > 4.0:
            return n
    return 0

@guvectorize([(complex64[:], int32[:], int32[:])], '(n),()->(n)',target='parallel')
def mandelbrot_numpy(c, maxit, output):
    maxiter = maxit[0]
    for i in range(c.shape[0]):
        output[i] = mandelbrot(c[i],maxiter)
        
def mandelbrot_set2(xmin,xmax,ymin,ymax,width=256,height=256,maxiter=100):
    print("hi")
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
    print("yo")
    n3 = mandelbrot_numpy(c,maxiter)
    print("fro")
    return n3.T

#%timeit mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-2.5, 2.5, -2.5, 2.5)

hi
yo
fro
CPU times: user 36.3 ms, sys: 3.13 ms, total: 39.4 ms
Wall time: 20.6 ms
hi
yo
fro
CPU times: user 4.84 ms, sys: 2.72 ms, total: 7.55 ms
Wall time: 4.1 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [34]:
import numpy

def mandelbrot(from_x, from_y, to_x, to_y, tile_size=256):
    xs = np.linspace(from_x, to_x, tile_size)
    ys = np.linspace(from_y, to_y, tile_size)
    
    limit = 4
    points = np.zeros((tile_size, tile_size))

    for j,x in enumerate(xs):
        for i,y in enumerate(ys):
            ax = x
            ay = y

            a1 = ax
            b1 = ay
            lp = 0

            while lp <= 60 and (a1 ** 2 + b1 ** 2) <= limit:
                lp += 1
                a2 = a1 ** 2 - b1 ** 2 + ax
                b2 = 2 * a1 * b1 + ay

                a1 = a2
                b1 = b2

                #print("a1:", a2)
            #print("lp:", ax, ay, lp)
            points[i,j] = lp
    return points

tsinfo = tileset_info()
%time mandelbrot(*hgut.tile_bounds(tsinfo, 0, 0, 0))

CPU times: user 695 ms, sys: 9 ms, total: 704 ms
Wall time: 763 ms


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [40]:
%%time 

import functools as ft
import numpy as np
import hgflask as hgf
import hgflask.client as hgc
import hgtiles.format as hgfo
import hgtiles.utils as hgut

# from https://plus.maths.org/content/computing-mandelbrot-set

def tiles(tsinfo, z, x, y):
    [from_x, from_y, to_x, to_y] = hgut.tile_bounds(tsinfo, z, x, y)
    tile_size = tsinfo['bins_per_dimension'] if 'bins_per_dimension' in tsinfo else 256
    limit = 4
    
    print("from_x", from_x, to_x, from_y, to_y)
    points = mandelbrot(from_x, from_y, to_x, to_y, tile_size)
    #points = mandelbrot_set2(from_x, to_x, from_y, to_y)
    print("points:", points)

    
    # ravel because the points are returned as a 2D array
    return hgfo.format_dense_tile(points.ravel())
    
tilesets = [{
    'uuid': 'a',
    'handlers': {
        'tileset_info': tileset_info,
        'tiles': ft.partial(hgut.tiles_wrapper_2d, 
                            tiles_function=ft.partial(tiles, tileset_info()))
    }
}]

server=hgf.start(tilesets)
#points = tiles(tsinfo, 1, 1, 1)

terminating: JtCGlH7EQdKBGs5R51tuGw


 * Running on http://0.0.0.0:58840/ (Press CTRL+C to quit)


sleeping
info: {'a': {'min_pos': [-2.5, -2.5], 'max_pos': [2.5, 2.5], 'bins_per_dimension': 256, 'max_width': 5, 'max_zoom': 20}}


127.0.0.1 - - [26/Sep/2018 23:01:59] "GET /api/v1/tileset_info/?d=a HTTP/1.1" 200 -


ret: 200 b'{\n  "a": {\n    "bins_per_dimension": 256, \n    "max_pos": [\n      2.5, \n      2.5\n    ], \n    "max_width": 5, \n    "max_zoom": 20, \n    "min_pos": [\n      -2.5, \n      -2.5\n    ]\n  }\n}\n'
returning
CPU times: user 22.3 ms, sys: 32.2 ms, total: 54.5 ms
Wall time: 288 ms


In [41]:
tilesets[0]['handlers']['tiles'](0,0,0)

TypeError: tiles_wrapper_2d() got multiple values for argument 'tiles_function'

In [36]:
server.tiles('a', 0, 0, 0)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [18]:
import json

conf = hgc.HiGlassConfig()
view = conf.add_view()
view.add_track('a', 'heatmap', 'center', 
               server=server.api_address,
               height=200, options={
                   'colorRange': ['white', 'black']
               })
view.add_track(None, 'top-axis', 'top')
view.add_track(None, 'left-axis', 'left')

#print(hgc.to_json_string())
import higlass_jupyter
higlass_jupyter.HiGlassDisplay(viewconf=conf.to_json_string(), 
                               hg_options='{"bounded": false, "renderer": "canvas"}')

A Jupyter Widget

tile_size 256
CPU times: user 718 ms, sys: 14.7 ms, total: 733 ms
Wall time: 814 ms


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])